In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import datetime
import sys


# new import added by Lewis
import math


In [46]:
# directory with interim charles river data results
results_file = "../data/charles_river_samples_interim/results_merged_cleaned.csv"

In [47]:
results = pd.read_csv(results_file)

In [97]:
for timevalues in ( results['Date_Collected_Time_Collected'] ).iteritems():
    if( len(timevalues[1])< 19 ):
        print( timevalues )

(6406, '2006-08-03 nan')
(6448, '2016-07-05 nan')
(6449, '2016-07-05 nan')
(6450, '2016-07-05 nan')
(13941, '2010-07-07 nan')
(13942, '2010-07-07 nan')
(13943, '2010-07-07 nan')
(13944, '2010-07-07 nan')
(13945, '2010-07-14 nan')
(13946, '2010-07-14 nan')
(13947, '2010-07-14 nan')
(13948, '2010-07-14 nan')
(13949, '2010-07-21 nan')
(13950, '2010-07-21 nan')
(13951, '2010-07-21 nan')
(13952, '2010-07-21 nan')
(13953, '2010-07-28 nan')
(13954, '2010-07-28 nan')
(13955, '2010-07-28 nan')
(13956, '2010-07-28 nan')
(13957, '2010-08-04 nan')
(13958, '2010-08-04 nan')
(13959, '2010-08-04 nan')
(13960, '2010-08-04 nan')
(13961, '2010-08-11 nan')
(13962, '2010-08-11 nan')
(13963, '2010-08-11 nan')
(13964, '2010-08-11 nan')
(13965, '2010-08-19 nan')
(13966, '2010-08-19 nan')
(13967, '2010-08-19 nan')
(13968, '2010-08-19 nan')
(13969, '2010-08-26 nan')
(13970, '2010-08-26 nan')
(13971, '2010-08-26 nan')
(13972, '2010-08-26 nan')
(13973, '2010-09-09 nan')
(13974, '2010-09-09 nan')
(13975, '2010-09

In [66]:
# starting with code to understand the data ...

In [67]:
# research 
results

,Date_Collected_Time_Collected,Activity_ID,Lab_ID,Site_ID,Project_ID,Actual_Result,Actual_Result_Unit_ID,Result_Sample_Fraction,Reporting_Result,Reporting_Result_Unit_ID,...,Percent_RPD,QAQC_Status,Data_Type_Name,Media_Type_Name,Media_Subdivision_Name,Component_Name,Activity_Type,Actual_Result_Type,Reporting_Result_Type,Relative_Depth_Name
0,2016-04-12 05:58:00,720160412ROV1DTH01,NaN,ROV1,7,0.67,5,NaN,0.67,5,...,372.0,Final-Accepted,Non-critical,Water,Surface Water,Depth,Field Msr/Obs,Actual,Actual,Surface
1,2016-05-17 05:58:00,720160517ROV1DTH01,NaN,ROV1,7,0.46,5,NaN,5.50,5,...,2.0,Final-Accepted,Non-critical,Water,Surface Water,Depth,Field Msr/Obs,Actual,Actual,Surface
2,2016-07-19 06:45:00,720160719ROV1DTH01,NaN,ROV1,7,0.33,5,NaN,0.33,5,...,2.0,Final-Accepted,Non-critical,Water,Surface Water,Depth,Field Msr/Obs,Actual,Actual,Surface
3,2016-07-19 07:00:00,720160719ROV2DTH01,NaN,ROV2,7,0.5,5,NaN,0.50,5,...,4.0,Final-Accepted,Non-critical,Water,Surface Water,Depth,Field Msr/Obs,Actual,Actual,Surface
4,2016-08-16 07:17:00,720160816ROV1DTH01,NaN,ROV1,7,0.33,5,NaN,0.33,5,...,6.0,Final-Accepted,Non-critical,Water,Surface Water,Depth,Field Msr/Obs,Actual,Actual,Surface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46445,2008-08-11 09:46:39,FLG200808113BUTemp03,NaN,3BU,3,20.36,4,Total,20.36,4,...,NaN,Preliminary,Critical,Water,Surface Water,"Temperature, water",Field Msr/Obs,Actual,Actual,Bottom
46446,2008-08-11 09:15:20,FLG200808114LONGTemp02,NaN,4LONG,3,21.33,4,Total,21.33,4,...,NaN,Preliminary,Critical,Water,Surface Water,"Temperature, water",Field Msr/Obs,Actual,Actual,Bottom
46447,2008-08-11 09:15:31,FLG200808114LONGTemp03,NaN,4LONG,3,21.33,4,Total,21.33,4,...,NaN,Preliminary,Critical,Water,Surface Water,"Temperature, water",Field Msr/Obs,Actual,Actual,Bottom
46448,2008-08-11 09:21:29,FLG200808114LONGTemp05,NaN,4LONG,3,21.31,4,Total,21.31,4,...,NaN,Preliminary,Critical,Water,Surface Water,"Temperature, water",Field Msr/Obs,Actual,Actual,Bottom


In [50]:
# research:
# all 'Media_Subdivision_Name' values are surface water
pd.unique(results['Media_Subdivision_Name'])

array(['Surface Water'], dtype=object)

In [51]:
# research
# many different types of measurements are in 'Component_Name'
pd.unique(results['Component_Name'])

array(['Depth', 'Temperature, water', 'Escherichia coli', 'Enterococcus',
       'Fecal coliform', 'Inorganic nitrogen (nitrate and nitrite)',
       'Phosphorus', 'Total suspended solids', 'Chlorophyll a',
       'Pheophytin a', 'Nitrogen', 'Ammonia', 'Orthophosphate', 'Sodium',
       'Conductivity', 'Surfactants, anionic', 'Nitrite', 'Nitrate',
       'Chlorine', 'Ammonium', 'Barometric pressure',
       'Dissolved oxygen saturation', 'Dissolved oxygen (DO)', 'Salinity',
       'Algae, blue-green (phylum cyanophyta) density', 'pH',
       'Specific conductance'], dtype=object)

In [53]:
# from results table, only pull out the records (rows) for water temperature, and only pull out a few fields (columns)

results_temperature = results[results['Component_Name']=='Temperature, water'][['Date_Collected_Time_Collected', 'Reporting_Result']]
results_temperature

,Date_Collected_Time_Collected,Reporting_Result
6657,2016-04-12 05:58:00,14.50
6658,2016-05-17 05:58:00,15.00
6659,2016-09-20 06:00:00,20.50
6660,2016-09-20 05:25:00,21.30
6661,2016-11-15 06:15:00,14.80
...,...,...
46445,2008-08-11 09:46:39,20.36
46446,2008-08-11 09:15:20,21.33
46447,2008-08-11 09:15:31,21.33
46448,2008-08-11 09:21:29,21.31


In [86]:
len('2016-04-12 05:58:00')

19

In [96]:
for timevalues in ( results_temperature['Date_Collected_Time_Collected'] ).iteritems():
    if( len(timevalues[1])< 19 ):
        print( timevalues )

(13941, '2010-07-07 nan')
(13942, '2010-07-07 nan')
(13943, '2010-07-07 nan')
(13944, '2010-07-07 nan')
(13945, '2010-07-14 nan')
(13946, '2010-07-14 nan')
(13947, '2010-07-14 nan')
(13948, '2010-07-14 nan')
(13949, '2010-07-21 nan')
(13950, '2010-07-21 nan')
(13951, '2010-07-21 nan')
(13952, '2010-07-21 nan')
(13953, '2010-07-28 nan')
(13954, '2010-07-28 nan')
(13955, '2010-07-28 nan')
(13956, '2010-07-28 nan')
(13957, '2010-08-04 nan')
(13958, '2010-08-04 nan')
(13959, '2010-08-04 nan')
(13960, '2010-08-04 nan')
(13961, '2010-08-11 nan')
(13962, '2010-08-11 nan')
(13963, '2010-08-11 nan')
(13964, '2010-08-11 nan')
(13965, '2010-08-19 nan')
(13966, '2010-08-19 nan')
(13967, '2010-08-19 nan')
(13968, '2010-08-19 nan')
(13969, '2010-08-26 nan')
(13970, '2010-08-26 nan')
(13971, '2010-08-26 nan')
(13972, '2010-08-26 nan')
(13973, '2010-09-09 nan')
(13974, '2010-09-09 nan')
(13975, '2010-09-09 nan')
(13976, '2010-09-09 nan')
(13977, '2010-09-16 nan')
(13978, '2010-09-16 nan')
(13979, '201

In [72]:
# convert Date_Collected from string to datetime ( actually pandas._libs.tslibs.timestamps.Timestamp )

results_temperature['Date_Collected_Time_Collected'] = pd.to_datetime( results_temperature['Date_Collected_Time_Collected'] )
# type( results_temperature['Date_Collected_Time_Collected'][6657] )

ValueError: ('Unknown string format:', '2010-07-07 nan')

In [ ]:
# create new column:  which indicates the season collected

# season is an int indicating the year and season (first four digits are the year, and the last digit is the season)
# dates in December are treated as winter of the next year, so seasonal results are kept together
# winter ends with 1, spring ends with 2, summer ends with 3, autumn ends with 4

# This link discusses why meteorologists define the seasons this way
# https://www.ncei.noaa.gov/news/meteorological-versus-astronomical-seasons

def date_to_season(date):
    season = date.year*10; # first four digits will be the year
    
    if date.month == 12: # treat December as winter of the next year
       season += 11
    elif date.month < 3: # Jan and Feb are winter of the same year
        season += 1
    elif date.month < 6: # March, April, May are spring
        season += 2
    elif date.month < 9: # June, July, August are summer
        season += 3
    else:
        season += 4
        
    return season;

# results_temperature['Season_Collected'] = pd.to_datetime(results_temperature["Date_Collected"]).apply(date_to_season)
results_temperature['Season_Collected'] = (results_temperature["Date_Collected"]).apply(date_to_season)

In [ ]:
# research only

# list all seasons where this data was collected
# note:  no temperature measurements from Spring 2002 through Winter 2002/3
np.sort( results_temperature['Season_Collected'].unique() )

In [ ]:
# turns off warning described in below webpages (it appears to be a false error)
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
# https://www.dataquest.io/blog/settingwithcopywarning/
pd.options.mode.chained_assignment = None  # default='warn'

# add new column with year (for display)
results_temperature['Year'] = results_temperature.apply(lambda row: math.floor(row.Season_Collected/10), axis=1)

# turn warning back on:
pd.options.mode.chained_assignment = 'warn'

In [ ]:
# plotting temperature results values:
# reset index is necessary make the index go from 0 to (length-1)

# note the cycle observed here is due to the change of seasons throughout the year

res_plot = ( results_temperature.sort_values(by=['Date_Collected','Time_Collected']) )['Reporting_Result']
res_plot.reset_index(drop=True).plot()

In [ ]:
( results_temperature.sort_values(by=['Date_Collected','Time_Collected']) )

In [ ]:
# only use summertime data
# This is because people are more likely to go boating on and swimming in the
# Charles River during the summer, therefore summertime conditions are 
# anticipated to be more of interest to the CRWA than the other three seasons
results_temperature_summer = results_temperature[results_temperature['Season_Collected']%10 == 3]

In [ ]:
# create boxplot of summer data
# improvements wanted:  change title,
# space it out to show gap for missing 2002 data, create a linear regression showing the overall trend (if any)

# boxplot command
ax = results_temperature_summer.boxplot(by='Year', column='Reporting_Result', rot=90)

# labelling
ax.set_xlabel("Summer (June through August of Year)")
ax.set_ylabel("Temperature")
ax.set_title("")




In [ ]:
# plotting temperature results (summer only) values:
# reset index is necessary to make the index go from 0 to (length-1)
res_plot = ( results_temperature_summer.sort_values(by=['Date_Collected','Time_Collected']) )['Reporting_Result']
res_plot.reset_index(drop=True).plot()